In [58]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import logging
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
import random

# ロギングの設定
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Seleniumのオプションを設定
options = Options()
options.headless = True  # ヘッドレスモードでブラウザを起動
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_argument("--no-sandbox")
options.add_argument("--disable-web-security")  # 追加
options.add_argument("--disable-site-isolation-trials")  # 追加
options.add_argument("--disable-javascript")

# ChromeDriverのサービスを設定
service = ChromeService(ChromeDriverManager().install())

# Chromeブラウザを起動
driver = webdriver.Chrome(service=service, options=options)

# ページ全体のHTMLを取得するための関数
def get_page_html(driver):
    time.sleep(10)  # JavaScriptの実行を待つために10秒待機
    return driver.page_source

# 全角数字を半角数字に変換する関数
def convert_to_half_width(text):
    return text.translate(str.maketrans('０１２３４５６７８９', '0123456789'))

# データ抽出関数
def extract_data_from_page(html, seen_properties):
    soup = BeautifulSoup(html, 'html.parser')
    properties = soup.find_all('div', class_='p-property p-property--building js-block')
    logging.info(f"Found {len(properties)} properties on the current page")
    
    properties_list = []
    last_property_info = {}

    for index, prop in enumerate(properties):
        try:
            # 物件名が記載されているclassを選択
            title_element = prop.find('h2', class_='p-property__title--building')
            title = title_element.text.strip() if title_element else last_property_info.get('物件名')
            
            # 住所が記載されている要素を取得
            address_element = prop.find_all('dl', class_='p-property__information-hint')[0].find('dd')
            address = address_element.text.strip() if address_element else last_property_info.get('住所')

            # アクセス経路が記載されている要素を取得
            transport_elements = prop.find_all('dl', class_='p-property__information-hint')
            access_1 = transport_elements[1].find('dd').text.strip() if len(transport_elements) > 1 else last_property_info.get('アクセス1')
            
            # 築年数が記載されている要素を取得
            built_year_element = transport_elements[2].find('dd').text.strip() if len(transport_elements) > 2 else last_property_info.get('築年数')
            built_year_match = re.search(r'築(\d+)年', built_year_element) if built_year_element else None
            built_year = built_year_match.group(1) if built_year_match else last_property_info.get('築年数')
            
            # 各部屋の詳細情報を取得
            rooms = prop.find_all('div', class_='p-property__room--detailbox')
            for room in rooms:
                # 階数・部屋番号
                room_number = room.find('li', class_='p-property__room-number').text.strip() if room.find('li', class_='p-property__room-number') else None
                room_number = convert_to_half_width(room_number) if room_number else None

                # 賃料
                rent_price_element = room.find('b', class_='p-property__information-rent')
                rent_price_text = rent_price_element.text.strip().replace('万円', '').replace(',', '') if rent_price_element else None
                try:
                    rent_price = float(rent_price_text) if rent_price_text else None
                except ValueError:
                    rent_price = None  # 変換できない場合は None とする
                
                # 管理費
                rent_and_fee_element = room.find('li', class_='p-property__room-rent')
                if rent_and_fee_element:
                    management_fee_text = rent_and_fee_element.find('span').text.strip().replace('円', '').replace(',', '')
                    management_fee = int(management_fee_text) if management_fee_text.isdigit() else 0
                else:
                    management_fee = 0
                # 敷金の処理
                security_deposit_text = room.find_all('li', class_='p-property__room-keymoney')[0].text.strip().split('\n')[0].replace('敷', '')
                if 'なし' in security_deposit_text:
                    security_deposit = 0
                elif '万円' in security_deposit_text:
                    security_deposit = float(security_deposit_text.replace('万円', '').replace(',', '')) * 10000
                elif 'ヶ月' in security_deposit_text:
                    security_deposit = float(security_deposit_text.replace('ヶ月', '')) * float(rent_price)*10000
                else:
                    security_deposit = 0

                # 礼金の処理
                key_money_element = room.find('li', class_='p-property__room-keymoney')
                if key_money_element:
                    span_element = key_money_element.find('span')
                    key_money_text = span_element.text.strip() if span_element else ''
                    if 'なし' in key_money_text:
                        key_money = 0
                    elif '万円' in key_money_text:
                        key_money = float(key_money_text.replace('万円', '').replace(',', '')) * 10000
                    elif 'ヶ月' in key_money_text:
                        key_money = float(key_money_text.replace('ヶ月', '')) * float(rent_price)*10000
                    else:
                        key_money = 0
                else:
                    key_money = 0

                # 間取りの処理
                house_layout_element = room.find_all('li', class_='p-property__room-floorplan')[0].find('div', class_='p-property__floor')
                house_layout = house_layout_element.text.strip() if house_layout_element else None

                # 専有面積の処理
                exclusive_area_element = room.find_all('li', class_='p-property__room-floorplan')[0].find('span')
                exclusive_area = float(exclusive_area_element.text.strip().replace('m²', '')) if exclusive_area_element else None

                # 専有面積が40平方メートル以上の場合のみデータリストに情報を追加
                if exclusive_area and exclusive_area >= 40:
                    # 重複チェック
                    property_key = (address,rent_price, exclusive_area)
                    if property_key not in seen_properties:
                        seen_properties.add(property_key)
                        # 各部屋ごとのリンクを取得
                        room_link_element = room.find('a', class_='p-property__room-more-inner')
                        room_number_link = room_link_element['href'].split('/')[-2] if room_link_element else ''
                        room_link = f"https://www.athome.co.jp/chintai/{room_number_link}/"

                        properties_list.append({
                            '物件名': title,
                            'URL': room_link,
                            '住所': address,
                            'アクセス1': access_1,
                            '築年数': float(built_year) if built_year else None,
                            '賃料': float(rent_price) * 10000 if rent_price else None,
                            '管理費': int(management_fee) if management_fee else 0,
                            '間取り': house_layout,
                            '専有面積': exclusive_area,
                            '敷金': security_deposit,
                            '礼金': key_money,
                            '階/部屋番号': room_number
                        })
            
            # 最後の物件情報を保存
            last_property_info = {
                '物件名': title,
                '住所': address,
                'アクセス1': access_1,
                '築年数': float(built_year) if built_year else None
            }

        except Exception as e:
            logging.error(f"Error processing property {index + 1}: {e}")

    return properties_list

# データリスト
all_properties_list = []
seen_properties = set()

# ページ番号をインクリメントしてデータを取得
page = 1
while True:
    url = f'https://www.athome.co.jp/chintai/tokyo/shinagawa-city/list/page{page}/'
    logging.info(f"Processing URL: {url}")
    driver.get(url)
    try:
        html_content = get_page_html(driver)
        logging.debug(f"HTML content of page {page}: {html_content[:500]}...")  # ログにHTMLの一部を出力
        properties_list = extract_data_from_page(html_content, seen_properties)
        if not properties_list:
            logging.info(f"No properties found on page {page}. Ending scraping.")
            break  # データが取得できなかった場合、ループを終了
        all_properties_list.extend(properties_list)
        logging.info(f"Page {page} processed successfully.")

        #if page % 10 == 0: #スクレイピング対策の対策
        #    wait_time = random.uniform(10, 20)  # 30秒から60秒の間でランダムな待ち時間
        #    logging.info(f"Waiting for {wait_time} seconds before continuing.")
        #    time.sleep(wait_time)

        page += 1

        # 次のページが存在するかをチェック
        soup = BeautifulSoup(html_content, 'html.parser')
        next_page_element = soup.find('link', {'rel': 'next'})
        if not next_page_element:
            logging.info(f"No next page found. Ending scraping at page {page}.")
            break

    except Exception as e:
        logging.error(f"Error processing page {page}: {e}")
        break

# データフレームに変換して表示
if all_properties_list:
    df = pd.DataFrame(all_properties_list)
    print(df)
else:
    logging.error("No properties were found or processed")

# ブラウザを終了
driver.quit()


2024-05-24 13:10:18,526 - INFO - ====== WebDriver manager ======
2024-05-24 13:10:18,865 - INFO - Get LATEST chromedriver version for google-chrome
2024-05-24 13:10:18,921 - INFO - Get LATEST chromedriver version for google-chrome
2024-05-24 13:10:18,969 - INFO - Driver [/Users/emiyoneda/.wdm/drivers/chromedriver/mac64/125.0.6422.78/chromedriver-mac-x64/chromedriver] found in cache
2024-05-24 13:10:22,178 - INFO - Processing URL: https://www.athome.co.jp/chintai/tokyo/shinagawa-city/list/page1/
2024-05-24 13:10:41,621 - INFO - Found 30 properties on the current page
2024-05-24 13:10:41,814 - INFO - Page 1 processed successfully.
2024-05-24 13:10:42,649 - INFO - Processing URL: https://www.athome.co.jp/chintai/tokyo/shinagawa-city/list/page2/
2024-05-24 13:11:01,118 - INFO - Found 30 properties on the current page
2024-05-24 13:11:01,493 - INFO - Page 2 processed successfully.
2024-05-24 13:11:02,997 - INFO - Processing URL: https://www.athome.co.jp/chintai/tokyo/shinagawa-city/list/pag

                       物件名                                           URL  \
0                コーポ中村 2階建  https://www.athome.co.jp/chintai/1061852294/   
1              アメニティ５５ 8階建  https://www.athome.co.jp/chintai/6982403637/   
2        Ｆｅｌｉｃｅ（ペット共生） 3階建  https://www.athome.co.jp/chintai/1066571194/   
3            グラン　グリシーヌ 3階建  https://www.athome.co.jp/chintai/1023795596/   
4          Ａｎｇｅｒｓ　Ｄｅｕｘ 3階建  https://www.athome.co.jp/chintai/1022615896/   
..                     ...                                           ...   
89        コルテージア　コート－Ｋ 3階建  https://www.athome.co.jp/chintai/1024150896/   
90                高月ビル 3階建  https://www.athome.co.jp/chintai/1027355479/   
91  パークハウス旗の台六丁目 地上3階地下1階建  https://www.athome.co.jp/chintai/1017470995/   
92            ボヌール武蔵小山 3階建  https://www.athome.co.jp/chintai/1024151096/   
93   品川区 南大井４丁目 （立会川駅） 7階建  https://www.athome.co.jp/chintai/6982113955/   

           住所                アクセス1   築年数        賃料    管理費    間取り    専有面積  \
0   品川区西品川３

In [59]:
# データフレームに変換して表示
df = pd.DataFrame(all_properties_list)
df.head(50)

,物件名,URL,住所,アクセス1,築年数,賃料,管理費,間取り,専有面積,敷金,礼金,階/部屋番号
0,コーポ中村 2階建,https://www.athome.co.jp/chintai/1061852294/,品川区西品川３丁目,ＪＲ山手線 「大崎」駅 徒歩6分,32.0,220000.0,3000,2LDK,71.81,220000.0,220000.0,201
1,アメニティ５５ 8階建,https://www.athome.co.jp/chintai/6982403637/,品川区大井１丁目,ＪＲ京浜東北線 「大井町」駅 徒歩3分,28.0,200000.0,30000,2LDK,55.07,200000.0,0.0,201
2,Ｆｅｌｉｃｅ（ペット共生） 3階建,https://www.athome.co.jp/chintai/1066571194/,品川区小山４丁目,東急目黒線 「武蔵小山」駅 徒歩6分,2.0,216000.0,8000,2LDK,52.75,108000.0,216000.0,101
3,グラン グリシーヌ 3階建,https://www.athome.co.jp/chintai/1023795596/,品川区中延５丁目,東急大井町線 「荏原町」駅 徒歩4分,7.0,141000.0,5000,1LDK,44.84,141000.0,211500.0,102
4,Ａｎｇｅｒｓ Ｄｅｕｘ 3階建,https://www.athome.co.jp/chintai/1022615896/,品川区大井５丁目,ＪＲ横須賀線 「西大井」駅 徒歩11分,3.0,137000.0,7000,1LDK,41.25,137000.0,205500.0,101
5,ジェイグランコート品川西大井ウエスト 5階建,https://www.athome.co.jp/chintai/1090346586/,品川区西大井２丁目,ＪＲ埼京線 「西大井」駅 徒歩7分,5.0,170000.0,20000,2LDK,40.26,0.0,0.0,108
6,Ｊ．ＧＲＡＮ Ｃｏｕｒｔ 品川西大井ＷＥＳＴ 5階建,https://www.athome.co.jp/chintai/1057168884/,品川区西大井２丁目,ＪＲ横須賀線 「西大井」駅 徒歩7分,5.0,173000.0,20000,2LDK,40.26,0.0,0.0,211
7,Ｊ．ＧＲＡＮ Ｃｏｕｒｔ 品川西大井ＷＥＳＴ（ジェーグランコートシナガ 5階建,https://www.athome.co.jp/chintai/1051804386/,品川区西大井２丁目,ＪＲ横須賀線 「西大井」駅 徒歩7分,5.0,189000.0,20000,2LDK,40.26,0.0,0.0,301
8,品川区 西大井２丁目 （西大井駅） 5階建,https://www.athome.co.jp/chintai/1094031780/,品川区西大井２丁目,ＪＲ横須賀線 「西大井」駅 徒歩7分,5.0,174000.0,20000,2LDK,40.26,0.0,0.0,311
9,品川区 西大井２丁目 （西大井駅） 5階建,https://www.athome.co.jp/chintai/1094032280/,品川区西大井２丁目,ＪＲ横須賀線 「西大井」駅 徒歩7分,5.0,174000.0,20000,2LDK,41.11,0.0,0.0,316


In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   物件名     94 non-null     object 
 1   URL     94 non-null     object 
 2   住所      94 non-null     object 
 3   アクセス1   94 non-null     object 
 4   築年数     94 non-null     float64
 5   賃料      94 non-null     float64
 6   管理費     94 non-null     int64  
 7   間取り     94 non-null     object 
 8   専有面積    94 non-null     float64
 9   敷金      94 non-null     float64
 10  礼金      94 non-null     float64
 11  階/部屋番号  94 non-null     object 
dtypes: float64(5), int64(1), object(6)
memory usage: 8.9+ KB


In [62]:
#DBファイルとして保存する関数
def save_to_database(quotes, db_name=None):
    if db_name is None:
        date_str = datetime.now().strftime("%Y%m%d_%H%M%S")
        db_name = f'quotes_{date_str}.db'
    
    try:
        df = pd.DataFrame(quotes)
        conn = sqlite3.connect(db_name)
        df.to_sql('quotes', conn, if_exists='append', index=False)
        conn.close()
        logging.info(f"Data successfully saved to {db_name}")
    except Exception as e:
        logging.error(f"Error saving data to database: {e}")

In [64]:
# データベース名を指定
db_name = 'estate_list_athome2.db'  # ここに実際のデータベースファイル名を入力してください

save_to_database(df, db_name)

2024-05-24 13:23:10,206 - INFO - Data successfully saved to estate_list_athome2.db
